<a href="https://colab.research.google.com/github/Neuralwood-Net/face-recognizer-9000/blob/main/cats_dogs_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cats and Dogs Neural Net Test

### Imports

In [4]:
import numpy as np
import pandas as pd

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

from google.cloud import storage

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Read and prepare the data

In [6]:
client = storage.Client.from_service_account_json("/content/drive/My Drive/## Project/TDT4173 Deep Learning Project-91d3b469375c.json")

bucket_name = "tdt4173-datasets"
bucket = client.get_bucket(bucket_name)

blob_name = "cats-vs-dogs/tensors/catsdogs_processed_64px_24946_horizontal.torch"
blob = bucket.get_blob(blob_name)
data_file = "/content/catsdogs_processed_64px_24946_horizontal.torch"
blob.download_to_filename(data_file)

In [7]:
data = torch.load(data_file)

(
    X_train,
    X_val_test,
    y_train,
    y_val_test
 ) = train_test_split(data["x"], data["y"], test_size=0.30)

(
    X_val,
    X_test,
    y_val,
    y_test
 ) = train_test_split(X_val_test, y_val_test, test_size=0.50)

In [8]:
y_train.shape, y_val.shape, y_test.shape

(torch.Size([17462]), torch.Size([3742]), torch.Size([3742]))

### Prepare the data as datasets in loaders

In [19]:
BATCH_SIZE = 32

X_train = X_train.view(-1, 1, 64, 64).float()
X_val = X_val.view(-1, 1, 64, 64).float()
X_test = X_test.view(-1, 1, 64, 64).float()

# Wrap the tensors in a wrapper
train = torch.utils.data.TensorDataset(X_train, y_train)
val = torch.utils.data.TensorDataset(X_val, y_val)
test = torch.utils.data.TensorDataset(X_test, y_test)

# Create the dataloader
train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=False)
val_loader = torch.utils.data.DataLoader(val, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

### Create functions for training, validation, and evaluation

In [13]:
def validate(model, val_loader, loss_func):
    correct = 0
    for batch_idx, (X_batch, y_batch) in enumerate(val_loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        output = model(X_batch.float())
        loss = loss_func(output, y_batch)

        predicted = torch.argmax(output, 1)
        correct += (predicted == y_batch).sum()

    acc = float(correct*100) / float(BATCH_SIZE*(batch_idx+1))

    return loss, acc

In [14]:
def fit(model, train_loader, val_loader):
    optimizer = torch.optim.Adam(model.parameters())
    loss_function = nn.CrossEntropyLoss()
    EPOCHS = 5
    EVAL_EVERY = 50
    model.train()

    for epoch in range(1, EPOCHS + 1):
        correct = 0
        it = tqdm(train_loader, desc=f"Epoch: {epoch}")
        for batch_idx, (X_batch, y_batch) in enumerate(it):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(X_batch.float())
            loss = loss_function(output, y_batch)
            loss.backward()
            optimizer.step()

            predicted = torch.argmax(output, 1)
            correct += (predicted == y_batch).sum()

            if batch_idx % EVAL_EVERY == 0:
                acc = float(correct*100) / float(BATCH_SIZE*(batch_idx+1))

                val_loss, val_acc = validate(model, val_loader, loss_function)


                it.set_postfix({
                    "Loss": f"{loss.item():.4f}",
                    "Acc": f"{acc:.2f}",
                    "Val loss": f"{val_loss.item():.4f}",
                    "Val acc": f"{val_acc:.2f}",
                })

In [15]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0

    for test_imgs, test_labels in test_loader:
        test_imgs, test_labels = test_imgs.to(device), test_labels.to(device)
        output = model(test_imgs.float())
        predicted = torch.argmax(output, dim=1)
        correct += (predicted == test_labels).sum()

    print(f"Test accuracy: {float(correct) / (len(test_loader)*BATCH_SIZE):.3f}%")

### Prepare and train the CNN

In [16]:
class CNN(nn.Module):
    size_after_conv = 16 * 16 * 64
    def __init__(self):
        super(CNN, self).__init__()
        self.features = nn.Sequential(   
            nn.Conv2d(1, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            # nn.Dropout(),
            nn.Conv2d(32, 32, kernel_size=5, padding=2),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Conv2d(32, 64, kernel_size=5, padding=2),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Dropout(),
        )
        self.classify = nn.Sequential(
            nn.Linear(self.size_after_conv, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, self.size_after_conv)
        x = self.classify(x)

        return F.log_softmax(x, dim=1)

In [17]:
cnn = CNN().to(device)
print(cnn)

CNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): ReLU()
    (9): Dropout(p=0.5, inplace=False)
  )
  (classify): Sequential(
    (0): Linear(in_features=16384, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [22]:
fit(cnn, train_loader, val_loader)


Epoch: 1: 100%|██████████| 546/546 [00:34<00:00, 15.95it/s, Loss=0.5142, Acc=76.09, Val loss=0.4695, Val acc=74.57]

Epoch: 2: 100%|██████████| 546/546 [00:34<00:00, 16.02it/s, Loss=0.4348, Acc=77.16, Val loss=0.5265, Val acc=75.45]

Epoch: 3: 100%|██████████| 546/546 [00:34<00:00, 16.05it/s, Loss=0.3952, Acc=77.89, Val loss=0.5745, Val acc=77.38]

Epoch: 4: 100%|██████████| 546/546 [00:34<00:00, 16.05it/s, Loss=0.4210, Acc=78.44, Val loss=0.3719, Val acc=78.02]

Epoch: 5: 100%|██████████| 546/546 [00:34<00:00, 16.04it/s, Loss=0.4340, Acc=79.50, Val loss=0.5959, Val acc=75.99]


In [23]:
evaluate(cnn, test_loader)

Test accuracy: 0.772%
